# Conditioned LSTM

In [1]:
import random

import numpy as np

import torch
from torch import nn
import torch.nn.functional as F

from sklearn.decomposition import PCA

import gensim
from gensim.models import KeyedVectors
from gensim.models.lsimodel import LsiModel
from gensim.corpora.dictionary import Dictionary

D:\giuse\Conda\envs\thesis\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [2]:
with open('data/tokenized.txt','r') as f:
    tokenized = eval(f.read())
    
with open('data/tokens.txt','r') as f:
    tokens = eval(f.read())

In [3]:
x_arr = np.load('data/x.npy')
y_arr = np.load('data/y.npy')

In [4]:
def get_batches(arr_x, arr_y, batch_size):
    
    pos = 0
    
    for n in range(batch_size, arr_x.shape[0], batch_size):
        x = arr_x[pos:n]
        y = arr_y[pos:n]
        pos = n
        yield x, y

## Word2Vec

In [5]:
w2v_model = gensim.models.KeyedVectors.load("w2v.model", mmap='r')

In [6]:
vocab_size, emdedding_size = w2v_model.wv.vectors.shape
vocab_size, emdedding_size

(17862, 100)

In [7]:
embedding_size = w2v_model.wv.vectors.shape[1]
embedding_size

100

In [8]:
w2v_tensors = torch.FloatTensor(w2v_model.wv.vectors)

## Topic Modelling

### Corpus

In [9]:
dct = Dictionary(tokenized)
dct.filter_extremes(no_below=5, no_above=0.3)

In [10]:
len(dct)

5485

In [11]:
print(dct)

Dictionary(5485 unique tokens: ['across', 'all', 'annotation', 'arabic', 'baselines']...)


In [12]:
corpus = [dct.doc2bow(a) for a in tokenized]

### Latent Semantic Analysis (LSA)

In [13]:
n_hidden = 256
batch_size = 64
n_layers = 3

In [14]:
lsi = LsiModel(corpus, id2word=dct, num_topics=n_hidden, decay=0.2)

In [15]:
lsi.show_topics(5)

[(0,
  '0.862*"e" + 0.293*"de" + 0.163*"d" + 0.125*"les" + 0.122*"la" + 0.121*"des" + 0.108*"l" + 0.095*"et" + 0.087*"s" + 0.081*"le"'),
 (1,
  '0.177*"word" + 0.151*"text" + 0.137*"learning" + 0.135*"using" + 0.134*"information" + 0.132*"it" + 0.130*"performance" + 0.126*"tasks" + 0.122*"training" + 0.121*"or"'),
 (2,
  '0.777*"word" + 0.237*"embeddings" + 0.202*"words" + -0.142*"text" + 0.107*"languages" + -0.100*"domain" + -0.096*"knowledge" + -0.094*"dataset" + -0.091*"question" + -0.083*"training"'),
 (3,
  '-0.663*"translation" + -0.271*"english" + -0.270*"machine" + -0.168*"nmt" + -0.158*"languages" + 0.126*"information" + -0.118*"parallel" + -0.117*"mt" + 0.110*"knowledge" + -0.096*"source"'),
 (4,
  '0.305*"corpus" + -0.185*"neural" + -0.183*"training" + 0.180*"languages" + -0.162*"translation" + 0.154*"speech" + -0.152*"tasks" + -0.144*"propose" + 0.135*"system" + -0.134*"sentence"')]

In [17]:
lsi.show_topic(3, topn=10)

[('translation', -0.663295596485134),
 ('english', -0.2709134713390895),
 ('machine', -0.27049498939311634),
 ('nmt', -0.16822649912604723),
 ('languages', -0.15800620384957956),
 ('information', 0.12646583717818807),
 ('parallel', -0.11846985826794525),
 ('mt', -0.11745536996348017),
 ('knowledge', 0.1103081521205139),
 ('source', -0.0955311876535538)]

In [18]:
trans_topics = np.transpose(lsi.projection.u)

### Principal Component Analysis (PCA)

In [19]:
pca_topics = PCA(n_components=n_layers*batch_size, svd_solver='full').fit_transform(trans_topics)

In [20]:
pca_trans = np.transpose(pca_topics)

## Models

### Conditioned LSTM

In [21]:
class ConditionedLSTM(nn.Module):
    
    def __init__(self, n_hidden=256, n_layers=2, drop_prob=0.2, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.emb_layer = nn.Embedding(vocab_size, embedding_size)

        # LSTM
        self.lstm = nn.LSTM(embedding_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        # fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        
        x = x.long()

        # pass input through embedding layer
        embedded = self.emb_layer(x)     
        
        # get outputs and new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)
        
        # pass through a dropout layer
        out = self.dropout(lstm_output)
        
        # flatten out
        out = out.reshape(-1, self.n_hidden) 

        # put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' initializes hidden state '''

        hidden = (torch.FloatTensor(pca_trans.reshape(self.n_layers, batch_size, self.n_hidden)),
                 torch.ones(self.n_layers, batch_size, self.n_hidden))

        return hidden

In [22]:
# instantiate the model
cond_lstm = ConditionedLSTM(n_hidden=n_hidden, n_layers=n_layers)

print(cond_lstm)

ConditionedLSTM(
  (emb_layer): Embedding(17862, 100)
  (lstm): LSTM(100, 256, num_layers=3, batch_first=True, dropout=0.2)
  (dropout): Dropout(p=0.2, inplace=False)
  (fc): Linear(in_features=256, out_features=17862, bias=True)
)


### Condtiioned LSTM + Word2Vec

In [ ]:
class CondLSTM_Word2Vec(nn.Module):
    
    def __init__(self, n_hidden=256, n_layers=2, drop_prob=0.2, lr=0.001):
        super().__init__()

        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.emb_layer = nn.Embedding.from_pretrained(w2v_tensors)

        # LSTM
        self.lstm = nn.LSTM(embedding_size, n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        # fully-connected layer
        self.fc = nn.Linear(n_hidden, vocab_size)      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
        
        x = x.long()

        # pass input through embedding layer
        embedded = self.emb_layer(x)     
        
        # get outputs and new hidden state from the lstm
        lstm_output, hidden = self.lstm(embedded, hidden)
        
        # pass through a dropout layer
        out = self.dropout(lstm_output)
        
        # flatten out
        out = out.reshape(-1, self.n_hidden) 

        # put "out" through the fully-connected layer
        out = self.fc(out)

        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' initializes hidden state '''

        hidden = (torch.FloatTensor(pca_trans.reshape(self.n_layers, batch_size, self.n_hidden)),
                 torch.ones(self.n_layers, batch_size, self.n_hidden))

        return hidden

In [ ]:
# instantiate the model
cond_lstm_w2v = CondLSTM_Word2Vec(n_hidden=n_hidden, n_layers=n_layers)

print(cond_lstm_w2v)

## Training

In [29]:
def train(model, epochs=10, batch_size=32, lr=0.001, clip=1, print_every=32):
    
    # optimizer
    opt = torch.optim.Adam(model.parameters(), lr=lr)
    
    # loss
    criterion = nn.CrossEntropyLoss()

    model.train()

    for e in range(epochs):
        
        batch = 0
        
        # initialize hidden state
        h = model.init_hidden(batch_size)
        
        for x, y in get_batches(x_arr, y_arr, batch_size):
            batch += 1
            
            # convert numpy arrays to PyTorch arrays
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            # detach hidden states
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            model.zero_grad()
            
            # get the output from the model
            output, h = model(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(-1).long())

            # back-propagate error
            loss.backward()

            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)

            # update weigths
            opt.step()    
            
            
            # show epoch - batch - loss every n batches
            if batch % print_every == 0:
                
                tot_batches = int(x_arr.shape[0] / batch_size)
            
                print("Epoch: {}/{} -".format(e+1, epochs),
                      "Batch: {}/{} -".format(batch, tot_batches),
                      "Loss: {}".format(loss))

In [24]:
epochs = 20
path = 'weights/cond_lstm.pt'
loss = 0.2

optimizer = torch.optim.Adam(cond_lstm.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [25]:
torch.save({
            'epoch': epochs,
            'model_state_dict': cond_lstm.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': criterion
            }, path)

D:\giuse\Conda\envs\thesis\lib\site-packages\torch\serialization.py:292: UserWarning: Couldn't retrieve source code for container of type CrossEntropyLoss. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


In [30]:
train(cond_lstm, batch_size=batch_size, epochs=epochs, print_every=5)

Epoch: 1/20 - Batch: 5/91 - Loss: 6.9388556480407715
Epoch: 1/20 - Batch: 10/91 - Loss: 6.793819427490234
Epoch: 1/20 - Batch: 15/91 - Loss: 6.919086933135986
Epoch: 1/20 - Batch: 20/91 - Loss: 6.97359561920166
Epoch: 1/20 - Batch: 25/91 - Loss: 6.977308750152588
Epoch: 1/20 - Batch: 30/91 - Loss: 6.9340291023254395
Epoch: 1/20 - Batch: 35/91 - Loss: 6.987171649932861
Epoch: 1/20 - Batch: 40/91 - Loss: 6.788027763366699
Epoch: 1/20 - Batch: 45/91 - Loss: 6.8932647705078125
Epoch: 1/20 - Batch: 50/91 - Loss: 6.909572601318359
Epoch: 1/20 - Batch: 55/91 - Loss: 6.885960578918457
Epoch: 1/20 - Batch: 60/91 - Loss: 6.820892810821533
Epoch: 1/20 - Batch: 65/91 - Loss: 6.857855319976807
Epoch: 1/20 - Batch: 70/91 - Loss: 6.941005706787109
Epoch: 1/20 - Batch: 75/91 - Loss: 7.204423427581787
Epoch: 1/20 - Batch: 80/91 - Loss: 6.836722373962402
Epoch: 1/20 - Batch: 85/91 - Loss: 6.921016216278076
Epoch: 1/20 - Batch: 90/91 - Loss: 6.750144004821777
Epoch: 1/20 - Loss: 8.968854904174805
Epoch: 

## Generation

In [31]:
def word_to_id(word):
    return w2v_model.wv.key_to_index[word]

def id_to_word(id):
    return w2v_model.wv.index_to_key[id]

In [32]:
# predict next token
def predict(model, t, h=None): # default value as None for first iteration
         
    # tensor inputs
    x = np.array([[word_to_id(t)]])
    inputs = torch.from_numpy(x)

    # detach hidden state from history
    h = tuple([each.data for each in h])

    # get the output of the model
    out, h = model(inputs, h)

    # get the token probabilities
    p = F.softmax(out, dim=1).data
    
    p = p.numpy()
    p = p.reshape(p.shape[1],)

    # get indices of top n values
    top_ids = p.argsort()[-5:][::-1]

    # sample id of next word from top n values
    next_id = top_ids[random.sample([0,1,2,3,4],1)[0]]

    # return the value of the predicted word and the hidden state
    return id_to_word(next_id), h

In [33]:
gen_batch_size = 1

In [34]:
# PCA for generation batch size
gen_pca_topics = PCA(n_components=n_layers * gen_batch_size, svd_solver='full').fit_transform(trans_topics)
gen_pca_trans = np.transpose(gen_pca_topics)

In [37]:
# function to generate text
def generate(model=cond_lstm, n=10, prompt='in this paper'):
    
    model.eval()
    
    h = (torch.FloatTensor(gen_pca_trans.reshape(n_layers, gen_batch_size, n_hidden)),
         torch.ones(n_layers, gen_batch_size, n_hidden))

    words = prompt.split()

    # predict next token
    for t in prompt.split():
        token, h = predict(model, t, h)
    
    words.append(token)

    # predict subsequent tokens
    for i in range(n-1):
        token, h = predict(model, words[-1], h)
        words.append(token)

    return ' '.join(words)

In [40]:
generate(n=50)

'in this paper for both languages we present a novel approach in a large number in this study to end trainable and sequence learning we also present two novel approaches for the task of generating an grained and the corpus and a corpus based approach to the best process of our approach is'

In [44]:
generate(n=100, prompt='we use embeddings')

'we use embeddings on three benchmark data we present our results on two benchmark datasets for both language generation tasks and show that our approach can improve competitive results over state in both a small set on natural learning tasks and the lack of a variety to the task and a variety in the first task to predict the most complex information in order we present our system to predict the quality in this work the first study of generating word representations in this study is the first study to predict a corpus for a given corpus for this paper presents our approach'

## Evaluation